In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf

from scipy import io
from scipy.signal import butter, lfilter
import h5py
import random
import numpy as np
import os

import datetime
from time import gmtime, strftime, time

from nd_legacy import *

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def data():
    
    ecog_matfile = io.loadmat('test_ecog_data.mat')
    raw_data = ecog_matfile['raw_data']
    raw_acc = ecog_matfile['raw_acc']
    mv_acc = ecog_matfile['mv_acc']

    sampling_rate = 2000

    raw_data = raw_data[:,0:-10000]
    raw_acc = raw_acc[:,0:-10000]
    mv_acc = mv_acc[:,0:-10000]

    len_raw_data = raw_data.shape[1]
    
    raw_acc_f = butter_bandpass_filter(raw_acc, 0.1, 1, sampling_rate, order=3, how_to_filt = 'separately')

    idxs_train = np.arange(0,len_raw_data//3);
    idxs_val = np.arange(len_raw_data//3,len_raw_data//2);
    idxs_test = np.arange(len_raw_data//2,len_raw_data - 1000);

    raw_data_train = raw_data[4:8,idxs_train]
    raw_data_val = raw_data[4:8,idxs_val]
    raw_data_test = raw_data[4:8,idxs_test]

    mv_acc_train = raw_acc_f[:,idxs_train]
    mv_acc_val = raw_acc_f[:,idxs_val]
    mv_acc_test = raw_acc_f[:,idxs_test]

    def generate_slice(slice_len, pos=2, val=False):
        if val:
            X = raw_data_val
            y = mv_acc_val
        else:
            X = raw_data_train
            y = mv_acc_train

        len_X = X.shape[1]

        while True:
            slice_start = np.random.choice(len_X - slice_len)
            slice_end = slice_start + slice_len
            slice_mid = slice_start + slice_len//2
            slice_x = X[:,slice_start:slice_end].T

            if pos==0:
                slice_y = y[0,slice_start]
            elif pos==1:
                slice_y = y[0,slice_mid]
            else:            
                slice_y = y[0,slice_end]

            return slice_x, slice_y

    def train_generator(batch_size, slice_len, pos=2):
        while True:
            batch_x = []
            batch_y = []

            for i in range(0, batch_size):
                x, y = generate_slice(slice_len, pos, val=False)
                batch_x.append(x)
                batch_y.append(y)

            y = np.array(batch_y)
            x = np.array([i for i in batch_x])
            yield (x, y)
            
    def validation_generator(batch_size, slice_len, pos=2):
        while True:
            batch_x = []
            batch_y = []

            for i in range(0, batch_size):
                x, y = generate_slice(slice_len, pos, val=True)
                batch_x.append(x)
                batch_y.append(y)

            y = np.array(batch_y)
            x = np.array([i for i in batch_x])
            yield (x, y)

    return train_generator, validation_generator


In [3]:
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe
from hyperas.distributions import choice, uniform, conditional


Using TensorFlow backend.


In [4]:
from keras.layers import Conv1D, Dense, maximum, Dropout, Input, merge, GlobalMaxPooling1D, MaxPooling1D, Flatten, LSTM, BatchNormalization
from keras.models import Model, load_model
from keras.optimizers import RMSprop

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [13]:
def create_model(train_generator, validation_generator):

    ###
    
    slice_len = 2000

    nb_filters = 10
    kernel_size = 50

    pos = 2
    
    input_seq = Input(shape=(slice_len, 4))

    inputbn = BatchNormalization(axis = -1)(input_seq)

    convolved = Conv1D(nb_filters, kernel_size={{choice([50, 100, 200, 300, 500, 1000])}}, padding="same", activation='elu', kernel_initializer='orthogonal')(inputbn)
    convolvedbn = BatchNormalization(axis = -1)(convolved)
    pooled = MaxPooling1D(pool_size=4)(convolvedbn)

    convolved2 = Conv1D(nb_filters*4, kernel_size={{choice([50, 100, 200, 300, 500, 1000])}}, padding="same", activation='elu', kernel_initializer='orthogonal')(pooled)
    convolved2bn = BatchNormalization(axis = -1)(convolved2)
    pooled2 = MaxPooling1D(pool_size=4)(convolved2bn)

    convolved3 = Conv1D(nb_filters*16, kernel_size={{choice([50, 100, 200, 300, 500, 1000])}}, padding="same", activation='elu', kernel_initializer='orthogonal')(pooled2)
    convolved3bn = BatchNormalization(axis = -1)(convolved3)
    pooled3 = MaxPooling1D(pool_size=4)(convolved3bn)

    convolved4 = Conv1D(nb_filters*64, kernel_size={{choice([50, 100, 200, 300, 500, 1000])}}, padding="same", activation='elu', kernel_initializer='orthogonal')(pooled3)
    convolved4bn = BatchNormalization(axis = -1)(convolved4)

    flat = GlobalMaxPooling1D()(convolved4bn)

    dense1 = maximum([Dense(1000, activation='linear', kernel_initializer='glorot_normal')(flat) for _ in range(3)])
    dense1 = BatchNormalization()(dense1)
    dense1do = Dropout(0.2)(dense1)

    out = Dense(1, activation='linear')(dense1do) 
    model = Model(inputs=input_seq, outputs=out)

    ###

    model.compile(loss="mse", optimizer="adam", metrics=["mse"])
    
    ##

    #path_to_save_model = "1511_hyper_test"

    steps_per_epoch = 600
    validation_steps = 300
    epochs = 200
    patience = 20

    batch_size = 16

    #earlyStopping = EarlyStopping(monitor='val_loss', patience=patience, verbose=2, mode='auto')
    #checkpointer = ModelCheckpoint(path_to_save_model, monitor='val_loss', verbose=2,
    #                               save_best_only=True, mode='auto', period=1)

    model.fit_generator(train_generator(slice_len=slice_len, batch_size=batch_size), 
                        steps_per_epoch=steps_per_epoch, 
                        epochs=epochs, 
                        #callbacks=[earlyStopping, checkpointer], 
                        verbose=1, 
                        validation_steps=validation_steps, 
                        validation_data=validation_generator(slice_len=slice_len, batch_size=batch_size, pos=pos))

    score, acc = model.evaluate_generator(data_generator(slice_len=slice_len, batch_size=batch_size, 
                                                     pos=pos, val=True),500)

    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [14]:
train_generator, validation_generator = data()

best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='Untitled-1511-hyperopt-attempt')

print("Evaluation of best performing model:")

print(best_model.evaluate(validation_generator))

>>> Imports:
#coding=utf-8

try:
    import os
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from scipy import io
except:
    pass

try:
    from scipy.signal import butter, lfilter
except:
    pass

try:
    import h5py
except:
    pass

try:
    import random
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import os
except:
    pass

try:
    import datetime
except:
    pass

try:
    from time import gmtime, strftime, time
except:
    pass

try:
    from nd_legacy import *
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas.distributions import choice, uniform, conditional
except:
    pass

try:
    from keras.layers import Conv1D, Dense, maximum, Dropout, Input, merge, GlobalMaxPooling1D, MaxPooling1D, Flatten, LSTM, BatchNormalization
except:
    pass

try:

ResourceExhaustedError: OOM when allocating tensor with shape[1000]
	 [[Node: Variable_48/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable_48"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Variable_48, Const_53)]]

Caused by op u'Variable_48/Assign', defined at:
  File "/home/ubuntu/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/ubuntu/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/ubuntu/anaconda2/lib/python2.